### *Podemos rodar às segundas-feiras já que as partidas da liga são jogadas aos domingos*

In [1]:
import pandas as pd
from functions import clube_id_e_sessao, pega_dados, conecta_ao_db

In [2]:
sessao, club_id= clube_id_e_sessao()

Clube ID: 79085


### Informações gerais dos jogos

In [3]:
# Conecta ao banco de dados
db= conecta_ao_db('dados_sokker.db')
curs= db.cursor()

#Pega informações únicas das partidas já adicionadas para que não sejam inseridos valores duplicados na base
matches = pd.read_sql_query("SELECT * FROM matches", db)

ids= matches['matchID'].unique()
data_esperada= matches['dateExpected'].unique()
data_iniciada= matches['dateStarted'].unique()

In [4]:
# Id do clube a serem pegas as informações
id_clube= club_id

# Pegando informações das partidas retirando aquelas que já foram pegas
url= f'https://sokker.org/xml/matches-team-{club_id}.xml'
df_matches= pega_dados(url, sessao, search= 'match')

#Retirando valores nulos (partidas ainda não jogadas) e duplicados (partidas já adicionadas na base)
df_matches.dropna(inplace= True)

query= 'matchID not in @ids and dateExpected not in @data_esperada and dateStarted not in @data_iniciada'

# Retirando colunas que não salvaremos as informações
df_matches.drop(columns= ['homeTeamName', 'awayTeamName'], 
                inplace= True)

#df_matches= pd.concat([matches, df_matches])

df_matches= df_matches.query(query)
df_matches

,matchID,homeTeamID,awayTeamID,leagueID,round,season,week,day,dateExpected,dateStarted,homeTeamScore,awayTeamScore,supporters,weather,isFinished
34,42954130,30280,79085,200,0,0,1147,6,2025-07-04 18:22,2025-07-04 18:23,2,0,10084,5,1
35,42957573,82924,79085,51530,0,0,1148,0,2025-07-05 12:30,2025-07-05 12:30,8,0,7466,1,1
36,42740274,79085,79587,1349,5,71,1148,1,2025-07-06 21:59,2025-07-06 21:59,0,3,15855,5,1
37,42957569,79085,90579,51530,0,0,1148,2,2025-07-07 18:00,2025-07-07 18:00,0,3,2177,5,1
38,42957576,82354,79085,51530,0,0,1148,3,2025-07-08 19:30,2025-07-08 19:30,8,0,6183,4,1
39,42740276,124139,79085,1349,6,71,1148,4,2025-07-09 21:58,2025-07-09 21:59,2,0,18349,4,1
40,42957825,77240,79085,51530,0,0,1148,5,2025-07-10 19:30,2025-07-10 19:30,2,2,1925,3,1


In [5]:
if len(df_matches) > 0:
    
    # Adiciona dados à tabela
    df_matches.to_sql('matches', con=db, if_exists='append', index= False)

In [6]:
#df_matches = pd.read_sql_query("SELECT * FROM matches", db)
len(df_matches)

7

### Estatísticas dos jogos

In [7]:
#Pega informações únicas das estatísticas já adicionadas para que não sejam inseridos valores duplicados na base
stats = pd.read_sql_query("SELECT * FROM matchStats", db)

ids= stats['matchID'].unique()
team= stats['teamID'].unique()
terco_defensivo= stats['timeOnHalf'].unique()
posse_de_bola= stats['timePossession'].unique()

In [8]:
#lista que guardará as estatísticas de cada partida
list_stats= list()

# Pegando estatísticas

for ID in df_matches['matchID'].unique():
    
    url= f'https://sokker.org/xml/match-{ID}.xml'
    df= pega_dados(url, sessao, search= 'teamStats')
    df['matchID']= ID
    list_stats.append(df)
    


df_stats= pd.concat([stats for stats in list_stats])

# Retirando valores nulos (partidas ainda não jogadas) e duplicados (partidas já adicionadas na base)

df_stats.dropna(inplace= True)

query= 'matchID not in @ids and teamID not in @team and timeOnHalf not in @terco_defensivo and timePossession not in @posse_de_bola'

df_stats= df_stats.query(query)

df_stats

,teamID,timeOnHalf,timePossession,offsides,shoots,fouls,yellowCards,redCards,tacticName,ratingScoring,ratingPassing,ratingDefending,matchID
0,30280,26786,39136,3,17,5,2,0,SV Saints,9,8,8,42954130
1,79085,43456,31106,0,6,2,2,1,4-4-2c,6,7,7,42954130
0,82924,20562,38377,0,25,3,1,0,4-2-3-1 B+ D,10,11,10,42957573
1,79085,49400,31585,5,2,2,0,0,4-3-3a,8,9,11,42957573
0,79085,44359,31207,1,3,3,0,1,3-4-3aQJ,8,9,11,42740274
1,79587,25595,38747,1,14,2,0,0,bola 4312,10,9,10,42740274
0,79085,50189,36594,4,4,2,0,0,3-5-2aV,9,9,11,42957569
1,90579,21082,34677,0,11,3,0,0,DEFENSIVA ALA E,11,10,11,42957569
0,82354,33602,34501,0,14,2,0,0,5-3eRflügel,12,11,11,42957576
1,79085,36363,35464,0,4,1,0,0,3-5-2aV,9,9,12,42957576


In [9]:
# Adicionando informações se houver alguma nova
if len(df_stats) > 0:
    
    # Adiciona dados à tabela
    df_stats.to_sql('matchStats', con=db, if_exists='append', index= False)

In [10]:
# Conferindo
stats = pd.read_sql_query("SELECT * FROM matchStats", db)
len(stats)

1852

### Estatísticas da Liga

In [11]:
#Pega informações ligas já adicionadas para que não sejam inseridos valores duplicados na base
league_stats = pd.read_sql_query("SELECT * FROM leagueStats", db)

ids= league_stats['leagueID'].unique()
team= league_stats['teamID'].unique()
rodada=  league_stats['round'].unique()

In [12]:
#Pegar o id da liga desejada entrando na liga na página do sokker
league_id= 1349

url= f'https://sokker.org/xml/league-{league_id}.xml'
df_league_stats= pega_dados(url, sessao, search= 'teams', tree_top= True)

In [13]:
df_league_stats['leagueID']= league_id
df_league_stats

,teamID,round,points,wins,draws,losses,goalsScored,goalsLost,rankTotal,leagueID
0,23775,6,16,5,1,0,19,6,16513019005011,1349
1,79587,6,15,5,0,1,23,3,15520023005001,1349
2,78751,6,13,4,1,1,13,3,13510013004002,1349
3,6276,6,13,4,1,1,11,4,13507011004012,1349
4,122169,6,12,4,0,2,19,10,12509019004008,1349
5,124139,6,10,3,1,2,22,9,10513022003010,1349
6,23755,6,10,3,1,2,16,12,10504016003009,1349
7,78671,6,9,3,0,3,10,14,9496010003007,1349
8,89569,6,3,1,0,5,13,22,3491013001004,1349
9,79085,6,2,0,2,4,1,11,2490001000006,1349


In [14]:
# Adicionando informações 
if len(df_league_stats) > 0:
    
    # Adiciona dados à tabela
    df_league_stats.to_sql('leagueStats', con=db, if_exists='append', index= False)

league_stats = pd.read_sql_query("SELECT * FROM leagueStats", db)
len(league_stats)

### Liga informação geral

In [15]:
#Pega informações das ligas já adicionadas para que não sejam inseridos valores duplicados na base
leagues = pd.read_sql_query("SELECT * FROM league", db)

ids= leagues['leagueID'].unique()
league_name= leagues['name'].unique()

In [16]:
#Pegando dados da liga
df_league= pega_dados(url, sessao, search= 'info')

#Retirando coluna que não será utilizada
df_league.drop(columns= ['userID', 'round'], inplace= True)

#Retirando valores duplicados (se as estatísticas já tiverem sido adicionadas na base)
query= 'leagueID not in @ids and name not in @league_name'
df_league= df_league.query(query)

df_league

,leagueID,name,countryID,division,season,type,isOfficial,isCup


In [17]:
# Adicionando informações se for uma liga ainda não adicionada
if len(df_league) > 0:
    
    # Adiciona dados à tabela
    df_league.to_sql('league', con=db, if_exists='append', index= False)

In [18]:
league = pd.read_sql_query("SELECT * FROM league", db)
len(league)

4

In [19]:
# Salvando alterações
db.commit()

# Fechando conexão com a base de dados
db.close()